In [204]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="This pattern is interpreted as a regular expression, and has match groups")



np.random.seed(42)
df = pd.DataFrame({
    'User_ID': np.arange(1, 1001).reshape(1000),
    'Name': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', None, 'Eve'], size=1000),
    'Age': np.random.choice([25, 30, 35, 40, 45, np.nan, -1], size=1000),
    'Email': np.random.choice(['user@example.com', 'test@domain.com', '', np.nan, 'invalidemail'], size=1000),
    'Signup Date': np.random.choice(['2020-01-01', '2021-05-20', 'not_a_date', np.nan, '2022-03-15'], size=1000),
    'Country': np.random.choice(['USA', 'us', 'Canada', 'CANADA', 'UK', 'N/A', np.nan], size=1000),
    'Income': np.random.normal(50000, 15000, size=1000),
    'Notes': np.random.choice(['Good', 'Bad', 'Average', np.nan, '', ' '], size=1000),
    'Subscribed': np.random.choice(['yes', 'no', 'YES', 'No', 'n', 'Y', np.nan], size=1000)
})

In [205]:
print("Original shape:", df.shape)
### 1. Dropping duplicate rows
df = df.drop_duplicates()
print("After dropping duplicates:", df.shape)


Original shape: (1000, 9)
After dropping duplicates: (1000, 9)


In [206]:
### 2. Missing values
# Fill NaN values in 'Name' with 'Unknown'
df['Name'] = df['Name'].fillna('Unknown')

In [207]:
# Replace negative or NaN ages with median age
df['Age'] = df['Age'].replace(-1, np.nan)
df['Age'] = df['Age'].fillna(df['Age'].median())

In [208]:
# Drop rows with missing emails or empty emails
df['Email'] = df['Email'].replace(['', 'nan'], np.nan)
df = df.dropna(subset=['Email'])

In [209]:
### 3. Convert date column to datetime
df['Signup Date'] = pd.to_datetime(df['Signup Date'], errors='coerce')
df = df.dropna(subset=['Signup Date'])  # Drop rows where date conversion fail

In [210]:
### 4. Clean 'Country' column
df['Country'] = df['Country'].str.upper().str.strip()
df['Country'] = df['Country'].replace(['US'], 'USA')
df['Country'] = df['Country'].replace(['N/A', 'NONE', '', np.nan], 'UNKNOWN')


In [211]:
### 5. Clean 'Subscribed' column to boolean
df['Subscribed'] = df['Subscribed'].str.lower().str.strip()
df['Subscribed'] = df['Subscribed'].map({'yes': True, 'y': True, 'no': False, 'n': False})
df['Subscribed'] = df['Subscribed'].fillna(False) 

In [212]:
### 6. Clean 'Notes' column (strip whitespace, fill missing)
df['Notes'] = df['Notes'].astype(str).str.strip()
df['Notes'] = df['Notes'].replace({'nan': 'No Notes', '': 'No Notes'})
df['Notes'] = df['Notes'].fillna('No Notes')

In [213]:
### 7. 
df['Signup_Year'] = df['Signup Date'].dt.year
df['Is_USA'] = df['Country'] == 'USA'

In [214]:
### 8. Rename columns
df = df.rename(columns={
    'User_ID': 'UserID',
    'Signup Date': 'Signup_Date',
    'Income': 'Income_USD$',
})

In [215]:
### 9. Reorder columns
columns_order = ['UserID', 'Name', 'Age', 'Email', 'Signup_Date', 'Signup_Year',
                 'Country', 'Is_USA', 'Income_USD$', 'Subscribed', 'Notes']
df = df[columns_order]

In [216]:
### 10. Round Income 
df['Income_USD$'] = df['Income_USD$'].round()

In [217]:
### 11. Removing unnecessary rows
df['Name'] = df['Name'].str.lower().str.strip()
df['Email'] = df['Email'].str.lower().str.strip()
bad_rows = df[(~df['Email'].str.contains('@')) | (df['Name'] == 'unknown') | (~df['Email'].str.contains('.')) | (~df['Email'].str.contains(r'\.(com|ru|ge|us)$'))].index
df = df.drop(index=bad_rows)


In [ ]:
### 12. Capitalizing names
df['Name'] = df['Name'].str.capitalize()


In [ ]:
### 13. Sort Values by UserID
df = df.sort_values('UserID').reset_index(drop=True)

In [220]:
### Result
df.head(10)

,UserID,Name,Age,Email,Signup_Date,Signup_Year,Country,Is_USA,Income_USD$,Subscribed,Notes
0,1,David,35.0,test@domain.com,2021-05-20,2021,CANADA,False,34872.0,False,Average
1,6,Bob,40.0,user@example.com,2020-01-01,2020,USA,True,46048.0,True,Good
2,9,Charlie,35.0,test@domain.com,2020-01-01,2020,UNKNOWN,False,45058.0,True,No Notes
3,13,Eve,35.0,user@example.com,2022-03-15,2022,USA,True,57039.0,True,Bad
4,25,Eve,40.0,test@domain.com,2021-05-20,2021,NAN,False,59331.0,False,No Notes
5,29,Alice,35.0,user@example.com,2020-01-01,2020,CANADA,False,80184.0,False,No Notes
6,33,David,35.0,user@example.com,2021-05-20,2021,CANADA,False,52173.0,True,No Notes
7,34,David,35.0,user@example.com,2020-01-01,2020,UK,False,50359.0,True,No Notes
8,38,Charlie,35.0,test@domain.com,2021-05-20,2021,USA,True,72986.0,False,No Notes
9,41,Alice,35.0,user@example.com,2021-05-20,2021,CANADA,False,55515.0,False,Average
